## Prior to this notebook I completed the following:

### Task 1:

1: Created a new, empty GBQ dataset called msba_wedge

2: Downloaded a zip file of cleaned Wedge data from Moodle

3: Decompressed and uploaded the cleaned Wedge files to Google Storage Cloud using the Storage Cloud UI.

4: Connected to Google Storage Cloud from Google Big Query, reading in all 53 transaction files into a single transaction table within my msba_wedge dataset.

5: Applied Google's auto schema settings to determine field types and then validated them in GBQ's schema view.


## This Notebook handles the following tasks:

### Task 2:
1: Use Python to connect to my GBQ instance
    
2: Build a list of owners
        
3: Take a sample of the owners

4: Build a dataframe of all records for the sample of owners

5: Write that data to a .txt file


### Task 3:

6. Use Pandas Dataframes and SQL to build three summary tables:

    - Sales by Date and Hour
    - Sales by Owner by Year and Month
    - Sales by Product Description by Year and Month. 


7. Build a single SQLite Database containing the three summary files. 

In [1]:
# run pip install --upgrade google-cloud-BigQuery
# import random is to support random owner sample
# Sqlalchemy allows me to read gbq data into a dataframe

import pandas as pd
import random
import sqlalchemy
from google.cloud import bigquery
from google.oauth2 import service_account

# establishing google credentials and creating engine and client variables to use with sqlalchemy

credentials = service_account.Credentials.from_service_account_file('C:\\Users\\dakot\\Desktop\\Repo\\du_wedge_project\\ADA_2022_Wedge_Umbel\\dakota-umt-msba-dc288f16c43e.json')
project_id = 'dakota-umt-msba'

from sqlalchemy.engine import create_engine

from config import bigquery_uri
engine = create_engine(bigquery_uri,credentials_path ='C:\\Users\\dakot\\Desktop\\Repo\\du_wedge_project\\ADA_2022_Wedge_Umbel\\dakota-umt-msba-dc288f16c43e.json')
client = bigquery.Client(credentials = credentials,project = project_id)

In [2]:
# SQL Query that Generates list of Distinct Owners

owners = ("""
       SELECT DISTINCT(card_no)
       FROM msba_wedge.Wedge_Transactions 
       WHERE card_no <> 3""")


# Stores unique owners in a variable called results

results = client.query(owners)


In [3]:
# creating empty LIST of owners

owners = []


# Populating owner list

for row in results:
    owners.append(row[0])

In [4]:
# generates a list of 500 random owners. This sample size outputs between 200MB and 250MB of data.

sample_owners = random.sample(owners,500)


# Turns list into Tuple so that I can use it as a reference in my next SQL statement's WHERE clause
# This gives a dynamic random sample of users rather than hardcoding a static list of users

sample_owners_tuple = tuple(sample_owners)


In [5]:
# Assigning query to a variable allows for updates to "WHERE/IN" each time the random sample is updated
sample_owner_query = """
    SELECT * 
    FROM dakota-umt-msba.msba_wedge.Wedge_Transactions 
    WHERE card_no IN {};""".format(sample_owners_tuple)



In [6]:
# Executing SQL statment that lists ALL records for sample of owners and storing it to dataframe
df = pd.read_sql_query(sample_owner_query, con = engine)

In [7]:
# Export to .txt file in current working directory
df.to_csv('sample_owner_data.txt',sep='\t')

In [ ]:
# The following summary tables will populate a SQLite db

In [8]:
#Creating Summary Table 1 - Sales by Date and Hour
# I included total return/void spend and items 
# because the total spend was confusing to me without also seeing these datapoints

sales_date_hour_query = """
SELECT 
  CAST(datetime AS date) AS date
, EXTRACT(HOUR FROM datetime) AS hour
, COUNT(DISTINCT(trans_no)) as total_transactions
, ROUND(SUM(total),2) AS total_net_spend
, ROUND(SUM(CASE WHEN total >=  0 THEN total END),2) AS total_non_negative_spend
, ROUND(SUM(CASE WHEN total < 0 THEN total END),2) AS total_negative_spend
, COUNT(CASE WHEN trans_status IS NULL 
        OR trans_status = " "
        THEN 1 END) AS total_items_sold
, COUNT(CASE WHEN trans_status IS NOT NULL OR trans_status != " "
        THEN 1 END) AS total_items_returned_voided_or_other
 FROM `dakota-umt-msba.msba_wedge.Wedge_Transactions`
 GROUP BY date, hour
 ORDER BY date ASC
"""

In [9]:
#converting summary table 1 to dataframe
df_summary_table_1 = pd.read_sql_query(sales_date_hour_query, con = engine)

In [10]:
# Creating Summary Table 2 - Sales by Owner Year and Month
# I included negative and positive spend in addition to net spend  
# because the total spend and transactions was confusing to me without also seeing these datapoints.

owner_sales_year_month_query = """
SELECT 
  card_no AS card_no
, EXTRACT(YEAR FROM datetime) AS year
, EXTRACT(MONTH FROM datetime) AS month
, COUNT(DISTINCT(trans_no)) as total_transactions
, ROUND(SUM(total),2) AS total_net_spend
, ROUND(SUM(CASE WHEN total >=  0 THEN total END),2) AS total_non_negative_spend
, ROUND(SUM(CASE WHEN total < 0 THEN total END),2) AS total_negative_spend
, COUNT(CASE WHEN trans_status IS NULL 
        OR trans_status = " "
        THEN 1 END) AS total_items_sold
, COUNT(CASE WHEN trans_status IS NOT NULL OR trans_status != " "
        THEN 1 END) AS total_items_returned_voided_or_other
 FROM `dakota-umt-msba.msba_wedge.Wedge_Transactions`
 GROUP BY card_no, year,month
 ORDER BY total_net_spend DESC

"""

In [11]:
#converting summary table 2 to dataframe
df_summary_table_2 = pd.read_sql_query(owner_sales_year_month_query, con = engine)

In [12]:
# Creating Summary Table 3 - product sales by year and month
# I included negative and positive spend in addition to net spend  
# because the total spend and transactions was confusing to me without also seeing these datapoints.

product_sales_year_month_query = """

SELECT
 tran.upc AS upc
,tran.description AS description
,tran.department AS department_number
,dept.dept_name AS department_name
, EXTRACT(YEAR FROM tran.datetime) AS year
, EXTRACT(MONTH FROM tran.datetime) AS month
, COUNT(DISTINCT(tran.trans_no)) as total_transactions
, ROUND(SUM(tran.total),2) AS total_net_spend
, ROUND(SUM(CASE WHEN tran.total >=  0 THEN tran.total END),2) AS total_non_negative_spend
, ROUND(SUM(CASE WHEN tran.total < 0 THEN tran.total END),2) AS total_negative_spend
, COUNT(CASE WHEN tran.trans_status IS NULL 
        OR tran.trans_status = " "
        THEN 1 END) AS total_items_sold
, COUNT(CASE WHEN tran.trans_status IS NOT NULL OR tran.trans_status != " "
        THEN 1 END) AS total_items_returned_voided_or_other
 FROM `dakota-umt-msba.msba_wedge.Wedge_Transactions` tran
 JOIN `dakota-umt-msba.msba_wedge.dept_lookup` dept ON dept.department = tran.department
 GROUP BY upc,description,department_number,department_name,year,month
 
"""

In [13]:
#converting summary table 3 to dataframe
df_summary_table_3 = pd.read_sql_query(product_sales_year_month_query, con = engine)

In [18]:
# The following code creates a SQLite db containing the 3 summary tables from above

import sqlite3
from sqlite3 import Error

# Creating the database file in the current working directory
create_connection = (r"wedge_summary.db")

# Connecting to database
conn = sqlite3.connect("wedge_summary.db")
c = conn.cursor()

# Dropping tables if they already exist
drop_table1 = """ DROP TABLE IF EXISTS sales_date_hour """
c.execute(drop_table1)

drop_table2 = """ DROP TABLE IF EXISTS owner_sales_year_month """
c.execute(drop_table2)

drop_table3 = """ DROP TABLE IF EXISTS product_sales_year_month """
c.execute(drop_table3)

# Creating empty tables and defining schemas
t1 = """ CREATE TABLE sales_date_hour(
                      date TEXT
                    , hour INTEGER
                    ,total_transactions INTEGER
                    ,total_net_spend NUMERIC
                    ,total_non_negative_spend NUMERIC
                    ,total_negative_spend NUMERIC
                    ,total_items_sold INTEGER
                    ,total_items_returned_voided_or_other INTEGER
                    )"""
c.execute(t1)
print("Table 1 created successfully")

t2 = """ CREATE TABLE owner_sales_year_month(
                      card_no TEXT
                    , year INTEGER
                    , month INTEGER
                    ,total_transactions INTEGER
                    ,total_net_spend NUMERIC
                    ,total_non_negative_spend NUMERIC
                    ,total_negative_spend NUMERIC
                    ,total_items_sold INTEGER
                    ,total_items_returned_voided_or_other INTEGER
                    )"""
c.execute(t2)
print("Table 2 created successfully")




t3 = """ CREATE TABLE product_sales_year_month(
                      upc TEXT
                    , description TEXT
                    , department_number INTEGER
                    , department_name TEXT
                    , year INTEGER
                    , month INTEGER
                    ,total_transactions INTEGER
                    ,total_net_spend NUMERIC
                    ,total_non_negative_spend NUMERIC
                    ,total_negative_spend NUMERIC
                    ,total_items_sold INTEGER
                    ,total_items_returned_voided_or_other INTEGER
                    )"""
c.execute(t3)
print("Table 3 created successfully")



# write the data to a sqlite table
df_summary_table_1.to_sql('sales_date_hour', conn, if_exists='replace', index = False)

df_summary_table_2.to_sql('owner_sales_year_month', conn, if_exists='replace', index = False)

df_summary_table_3.to_sql('product_sales_year_month', conn, if_exists='replace', index = False)



#commiting changines and closing connection

conn.commit()
conn.close()

Table 1 created successfully
Table 2 created successfully
Table 3 created successfully
